# Prepare Data
Notebook reads .CSV for the LA parking citation data and prepares the data for use within the LA DOT Parking Meter Execute Dashboard by summarizing it. The data is then stored within a SQLite database for use within the website.

## 1 Import Dependencies

In [1]:
#- Import Dependencies
import pandas as pd
import os
import datetime as dt

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

## 2 Import CSV
Using Panadas, the CSV that has been downloaded from Kaggle is loaded into a DataFrame. The source file is located in folder on computer that is not part of the iCloud backup as it is large; 1.3 GB.

In [2]:
sourceFilePath = "/Users/thefixermac/Development_Data/Workshop/DataScience_Project_3/los-angeles-parking-citations/parking-citations.csv"  

print("Started reading of csv")
allCitations_df = pd.read_csv(sourceFilePath)


#- Display 
allCitations_df.head(20)

Started reading of csv


/Users/thefixermac/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0
5,1106226590,2015-09-15T00:00:00,19.0,NaN,NaN,CA,201507.0,NaN,CHEV,VN,GY,SAN PEDRO S/O BOYD,1A35W,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
6,1106500452,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,99999.0,99999.0
7,1106500463,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201602.0,NaN,TOYO,PA,BK,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,99999.0,99999.0
8,1106506402,2015-12-22T00:00:00,945.0,NaN,NaN,CA,201605.0,NaN,CHEV,PA,BR,721 S WESTLAKE,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,99999.0,99999.0
9,1106506413,2015-12-22T00:00:00,1100.0,NaN,NaN,CA,201701.0,NaN,NISS,PA,SI,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,99999.0,99999.0


In [3]:
#- Determine Number of records
allCitations_df.shape

(9389057, 19)

## 3 Prepare Date Column
Imported dataset contains column, "Issue Date", that is an object. Create a new column that is a date column which will be used for filtering.

In [4]:
#- Date Column Name
dateColumnName = "DateIssued"

#- Create Column
allCitations_df[dateColumnName] = pd.to_datetime(allCitations_df['Issue Date'])

#- Preview Dataset
allCitations_df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,DateIssued
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015-12-21
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015-12-21
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,2015-12-21
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2,2015-12-26
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,2015-09-15


## 4 Filter for Meters
The citation data contains all sorts of violations. It has been determined in order to only get records related to meters is to filter by the violation code of "88.13B+"; this is meter expiration. 

In [5]:
meterCitations_df = allCitations_df.loc[(allCitations_df["Violation code"] == '88.13B+'), :]

meterCitations_df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,DateIssued
530,4270903991,2015-12-30T00:00:00,807.0,WF192,NaN,CA,201605.0,NaN,AUDI,PA,GY,8330 3RD ST W,00451,54.0,88.13B+,METER EXP.,63.0,6449295.9,1849076.0,2015-12-30
531,4270904002,2015-12-30T00:00:00,815.0,BF430,NaN,CA,201609.0,NaN,FIAT,PA,RD,8000 BEVERLY BL,00451,54.0,88.13B+,METER EXP.,63.0,6451529.2,1850273.5,2015-12-30
551,4271419121,2015-12-30T00:00:00,1220.0,HO6630,NaN,CA,201610.0,NaN,MNNI,PA,BN,1600 VINE ST N,496R1,54.0,88.13B+,METER EXP.,63.0,6462771.4,1858859.1,2015-12-30
552,4271419132,2015-12-30T00:00:00,1223.0,HO6632,NaN,CA,201607.0,NaN,HOND,PA,GY,1610 VINE ST N,496R1,54.0,88.13B+,METER EXP.,63.0,6462771.2,1858927.1,2015-12-30
558,4271419191,2015-12-30T00:00:00,1244.0,HO966,NaN,CA,201604.0,NaN,FORD,VN,WT,1737 WHITLEY AV,496R1,54.0,88.13B+,METER EXP.,63.0,6460757.0,1859763.7,2015-12-30


In [6]:
#- Determine number of records within filtered dataframe
meterCitations_df.shape

(1620587, 20)

## 5 Prepare Output Database
Using SQLalchemy, define the schema of the summary database and then create database which is to be used to store the summarized data. 

#### 5.1 Delete Existing
First ensure that the existing database is deleted.

In [7]:
#- Output Database Name
outputDatabaseName = "parkingCitations.sqlite"

#- Delete Existing Database
outputDatabasePath = os.path.join(".", "Resources", outputDatabaseName)

#- Remove Existing File
fileExists = os.path.isfile(outputDatabasePath)

if fileExists:
    print(f"Output database exists, will delete. Path: {outputDatabasePath}")
    
    os.remove(outputDatabasePath)

else:
    print(f"Output database does not exist. Path: {outputDatabasePath}")
    

Output database exists, will delete. Path: ./Resources/parkingCitations.sqlite


#### 5.2 Create Database Schema
Prepare the sqlalchemy classes that represents the different table 

In [8]:
#- Create Table Schemas
Base = declarative_base()

class Agency(Base):
    __tablename__ = "Agency"
    Name = Column(String)
    AgencyID = Column(Integer, primary_key=True)
    
class AgencySummary(Base):
    __tablename__ = "AgencySummary"
    AgencySummary_PK = Column(Integer, primary_key=True)
    AgencyID_FK = Column(Integer)
    Count = Column(Integer)
    Year = Column(Integer)

class AgencyDayOfWeekSummary(Base):
    __tablename__ = "AgencyDayOfWeekSummary"
    AgencyDayOfWeekSummary_PK = Column(Integer, primary_key=True)
    AgencyID_FK = Column(Integer)
    Year = Column(Integer)
    Dow = Column(Integer)
    Count = Column(Integer)

class AgencyTimeOfDaySummary(Base):
    __tablename__ = "AgencyTimeOfDaySummary"
    AgencyTimeOfDaySummary_PK = Column(Integer, primary_key=True)
    AgencyID_FK = Column(Integer)
    Year = Column(Integer)
    Time = Column(Integer)
    Count = Column(Integer)

class AgencyTopMeters(Base):
    __tablename__ = "AgencyTopMeters"
    AgencyTopMeters_PK = Column(Integer, primary_key=True)
    AgencyID_FK = Column(Integer)
    MeterID = Column(String)
    Year = Column(Integer)
    Location = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Count = Column(Integer)


print("<- Completed creating database schema")

#### 5.2 Create Database
Create the file on disk

In [10]:
#- Create Database
engine = create_engine(f'sqlite:///Resources/{outputDatabaseName}', echo=False)

#- Create Tables
Base.metadata.create_all(engine)

print("<- Completed creating database.")

<- Completed creating database.


## 6 Summarize


#### 6.1 Summarize for Agency
Main function that is used prepare DataFrame then summarize.

In [11]:
def getAgencySummaryForYear(sourceCitations_df, agencyInfo, filterYear):
    ''' Calculates the summary information for the agency with the year provided and 
    stores results in database.
    
    Accepts : sourceCitations_df (DataFrame) contains records that are only citations
              agencyInfo (dictionary) metadata on the agency to process
                              name : name of the agency
                              id : unique identifier of the agency within the dataset
              filterYear (int) year to summarize data for
    
    Returns : nothing
    '''
    
    print(f'-> getAgencySummaryForYear. Agency ID: {agencyInfo["id"]} Year: {filterYear}')
     
          
    #- Filter for Agency and year
    filterCitations_df = sourceCitations_df.loc[((sourceCitations_df['Agency'] == agencyInfo["id"]) &
                                                 (sourceCitations_df[dateColumnName].dt.year == filterYear)), :]
    
    #- Update Database: AgencySummary
    session.add(AgencySummary(AgencyID_FK=agencyInfo["id"], Count=filterCitations_df.shape[0],
                             Year=filterYear))
    
          
    #- Summarize Day Of Week
    getAgencyDayOfWeekSummary(filterCitations_df, agencyInfo, filterYear)
    
          
    #- Summarize Time of Day
    getAgencyTimeOfDaySummary(filterCitations_df, agencyInfo, filterYear)
          
    
    #- Top Meters
    getAgencyTopMeters(filterCitations_df, agencyInfo, filterYear)
    
          
    

#### 6.2 Summarize for Day Of Week
Separate function to summarize for the day of week; using the created date column

In [12]:
def getAgencyDayOfWeekSummary(filterCitation_df, agencyInfo, filterYear):
    ''' Populates the AgencyDayOfWeekSummary; using the DataFrame provided. For each day of the
    week, 0 Monday, 1 Tuesday, determines the count and the updates the database.
    
    Accepts : filterCitation_df (DataFrame) contains only those records for the agency and year
              agencyInfo (Dictionary) metadata on agency to process
                              name: name of agency
                              id : unique identifier of the agency within the dataset
              filterYear (int) year to summary data for
        
    Returns : nothing
    '''
    
    print('-> getAgencyDayOfWeekSummary')

    
    for dayCounter in range(7):
        
        #- Filter for Day Of Week
        dayOfWeekFilter_df = filterCitation_df[filterCitation_df[dateColumnName].dt.dayofweek==dayCounter]
        
        #- Update Database: AgencyDayOfWeekSummary
        session.add(AgencyDayOfWeekSummary(AgencyID_FK=agencyInfo['id'], Year=filterYear, 
                                           Dow=dayCounter, Count=dayOfWeekFilter_df.shape[0]))
              

#### 6.3 Summarize for Time of Day
Separate function to summarize for the time of day; using the created date column.

In [13]:
def getAgencyTimeOfDaySummary(filterCitation_df, agencyInfo, filterYear):
    ''' Populates the AgencyTimeOfDaySummary table using the DataFrame provided.
    
    Accepts : filterCitation_df (DataFrame) contains only those records for the agency and year
              agencyInfo (Dictionary) metadata on agency to process
                              name: name of agency
                              id : unique identifier of the agency within the dataset
              filterYear (int) year to summary data for
        
    Returns : nothing
    '''
    
    print('-> getAgencyTimeOfDaySummary')
    
    startTime = 0
    
    for timeCounter in range(24):
        
        #- Determine End Time
        endTime = startTime + 100
        
        #- Filter for Time
        timeFilter_df = filterCitation_df[(filterCitation_df['Issue time'] >= startTime) & 
                                          (filterCitation_df['Issue time'] < endTime)]
        
        #- Update Database
        session.add(AgencyTimeOfDaySummary(AgencyID_FK=agencyInfo['id'], Year=filterYear, 
                                          Time=timeCounter, Count=timeFilter_df.shape[0]))
        
        #- Prepare Start Time for next loop
        startTime = endTime
        

#### 6.4 Top Meters
Get the top 100 meters for the district

In [14]:
def getAgencyTopMeters(filterCitation_df, agencyInfo, filterYear):
    ''' Populates the AgencyTopMeters table with the top (by count of citations) meters
    
    Accepts : filterCitation_df (DataFrame) contains only those records for the agency and year
              agencyInfo (Dictionary) metadata on agency to process
                              name: name of agency
                              id : unique identifier of the agency within the dataset
              filterYear (int) year to summary data for
        
    Returns : nothing
    '''
    
    print('-> getAgencyTopMeters')
    
    
    #- Group By Meter ID and get count
    meters_dfGroupBy = filterCitation_df.groupby('Meter Id').count()
    
    #- Sort 
    meterSorted_df = meters_dfGroupBy.sort_values('Ticket number', ascending=0)
    
    #- Get Top Meters
    meterCounter = 0
    
    for meterRow in meterSorted_df.itertuples():
        
        #- Get ID for Meter
        meterID = meterRow.Index
               
        #- Get Record for Meter; used for location
        individualMeter_df = filterCitation_df.loc[(filterCitation_df['Meter Id'] == meterID), :]
                
        #- Update Database
        session.add(AgencyTopMeters(AgencyID_FK = agencyInfo['id'], 
                                   MeterID = meterID,
                                   Year = filterYear,
                                   Location = individualMeter_df.iloc[0]['Location'],
                                   Latitude = individualMeter_df.iloc[0]['Latitude'],
                                   Longitude = individualMeter_df.iloc[0]['Longitude'],
                                   Count = meterRow[1]))
        
        #- Update Meter Count
        meterCounter = meterCounter + 1
        
        #- Check for Top 100
        if (meterCounter >= 100):
            break
    

In [ ]:
allCitations_df.shape[0]


## 7 Processing

#### 7.1 Prepare Years
The dataset contains records from 2015 until 2019. The executive dashboard is to only show full year data from 2015 until 2018.

In [15]:
#- Setup Years
# years = [
#     2015,
#     2016,
#     2017,
#     2018 ]

years = [2015, 2016]

print(years)

[2015, 2016]


#### 7.2 Prepare Agencies
The dataset contains data for multiple agencies and the focus of the executive dashboard is just the DOT districts. Create dictionary that contains both the name and the ID that is used to filter the DataFrame.

In [16]:
# agencies = [
#     { 
#         "name" : "Western",
#         "id" : 51
#     },
#     {
#         "name" : "Valley",
#         "id" : 53
#     },
#     {
#         "name" : "Hollywood",
#         "id" : 54
#     },
#     {
#         "name" : "Southern",
#         "id" : 55
#     },
#     {
#         "name" : "Central",
#         "id" : 56
#     }
# ]

agencies = [
    {
        "name" : "Hollywood",
        "id" : 54
    }

]

print(agencies)

[{'name': 'Hollywood', 'id': 54}]


#### 7.3 Process
Loop through the agencies and then the years to summarize the data.

In [17]:
#- Prepare Session
session = Session(bind=engine)


#- Loop Through Agencies
for agency in agencies:
    
    print(f'--> Processing agency: {agency["name"]}')
    
    
    #- Create Record in Agency Table
    session.add(Agency(Name=agency["name"], AgencyID=agency["id"]))
    
    
    #- Loop Through Years
    for year in years:
          
          #- Process for Year
          print(f'-> Processing year: {year}')
          
          getAgencySummaryForYear(meterCitations_df, agency, year)
          
          
          #- Save Changes To Database
          # Found that if the session was not commited, there would be a runtime error
          print(f'<- Commit changes for year: {year}')
          session.commit()
          

    
#- Commit Changes
session.commit()

print("<-- Completed saving session")

--> Processing agency: Hollywood
-> Processing year: 2015
-> getAgencySummaryForYear. Agency ID: 54 Year: 2015
-> getAgencyDayOfWeekSummary
-> getAgencyTimeOfDaySummary
-> getAgencyTopMeters
<- Commit changes for year: 2015
-> Processing year: 2016
-> getAgencySummaryForYear. Agency ID: 54 Year: 2016
-> getAgencyDayOfWeekSummary
-> getAgencyTimeOfDaySummary
-> getAgencyTopMeters
<- Commit changes for year: 2016
<-- Completed saving session


In [ ]:
years

In [ ]:
#- Agency ID
# 51: Western
# 53: Valley
# 54: Hollywood
# 55: Southern
# 56: Central



agencyID = 56


test_df = meterCitations_df.loc[(meterCitations_df["Agency"] == agencyID), :]

print(test_df.shape)
test_df.head()

In [ ]:
# Test filter by year
yearFilter = 2018



In [ ]:
import datetime as dt

test_df['DateColumn'] = pd.to_datetime(test_df['Issue Date'])

In [ ]:
filterDate_df = test_df[test_df['DateColumn'].dt.year == yearFilter]

filterDate_df.head()

In [ ]:
filterDate_df.shape

In [ ]:
#-- Day of week
# 0 - Monday
# 1- tuesday
# 2- Wednesday
# 3- Thursday
# 4- Friday
# 5- Saturday
# 6- Sunday
#
dayOfWeekFilter_df = test_df[test_df['DateColumn'].dt.dayofweek == 0]

print(dayOfWeekFilter_df.shape)
dayOfWeekFilter_df.head(20)

In [ ]:
#-- Time of day
timeFilter_df = test_df[(test_df['Issue time'] >= 1200) & (test_df['Issue time'] < 1300 ) ]

print(timeFilter_df.shape)
timeFilter_df.head(20)

In [ ]:
test_df['Issue time'].dtype

In [ ]:
#-- Top Meters
# Group by Meter Id and sort by most 

meters_dfGroupBy = test_df.groupby('Meter Id').count()



# print(type(meters_dfGroupBy))



# meters_dfGroupBy


testSort_df = meters_dfGroupBy.sort_values('Ticket number', ascending=0)

testSort_df.head(100)


In [ ]:
testSort_df = meters_dfGroupBy.sort_values('Ticket number', ascending=0)


    
    

In [ ]:
individual_df =  test_df.loc[(test_df['Meter Id'] == 'GA22'), :]

individual_df.iloc[0]['Location']




# theCounter = 0

# meterCitations_df.loc[(meterCitations_df["Agency"] == agencyID), :]
# for row in testSort_df.head().itertuples():
    
#     print(f'ID: {row.Index}')
#     print(f'count: {row[1]}')
    
#     theCounter = theCounter + 1
    
#     if (theCounter == 3):
#         break



In [ ]:
meterCitations_df.head()

In [ ]:
# test_df = meterCitations_df.loc[(meterCitations_df['Meter Id'] == '37'), :]

meterID = '37'

test2_df = meterCitations_df.loc[(meterCitations_df['Meter Id'] == meterID), :]

test2_df.head()

In [ ]:
test_df.shape